In [1]:
import spacy
import nltk

In [118]:
from spacy.cli import download

download("en_core_web_sm")

SystemExit: 1

c:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3259: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
f = open("bigram_stats.txt", encoding="utf-8", errors="ignore")
lines = f.readlines()

total = 0 
bigram_count = dict()
for line in lines:
    key, value = line.split(") ")
    key = key[1:]
    key = key.replace("'", "")
    first, second = key.split(", ")
    
    bigram_count[(first, second)] = int(value)
    total += int(value)

for key, value in bigram_count.items():
    bigram_count[key] /= total
    
print(bigram_count[(',', 'I')])
f.close()

0.003863573835380988


In [262]:
f = open("output.txt", encoding="utf-8", errors="ignore")
lines = f.readlines()
training = []
correct = []
for i in range(len(lines)):
    if i == 6000:
        break
    if i%4 == 2 or i%4 == 3:
        continue
    if i%4 == 0:
        training.append(lines[i].strip())
    else:
        correct.append(lines[i].strip())


In [245]:
POS_dict = {"ADJ": 0, "ADP": 1, "ADV": 2, "AUX": 3, "CONJ": 4, "CCONJ": 5, "DET": 6, "INTJ": 7,
            "NOUN": 8, "NUM": 9, "PART": 10, "PRON": 11, "PROPN": 12, "SCONJ": 13, "SYM": 14, 
            "VERB": 15, "X": 16, "SPACE": 17, "PUNCT": 18}
type_list = ["ADP", "NOUN"]
def get_pps(doc, _type, check_prev, test=False):
    "Function to get PPs from a parsed document."
    pps = []
    next = 0
    for i in range(len(doc)):
        if next != 0:
            next -= 1
            continue
        # Try this with other parts of speech for different subtrees.
        if doc[i].pos_ == _type:
            subtree = list(doc[i].subtree)
            pp = ' '
            ADP_list = []
            plural = 0
            next = len(subtree)

            word = ""
            if i != 0:
                if doc[i-1] in check_prev:
                    word = doc[i-1]
            if doc[i] in check_prev:
                word = doc[i]
            else:
                word = "null"


            if doc[i].tag_ == "NNS":
                plural = 1
            for j in range(len(subtree)):
                ADP_list.append(subtree[j].orth_)
            pp = ' '.join(ADP_list)
            input_feature = []

            for j in range(2, 0, -1):
                if i - j < 0:
                    input_feature.append(-1)
                else:
                    input_feature.append(POS_dict[doc[i - j].pos_])

            for j in range(1, 3):
                if i + j + next >= len(doc):
                    input_feature.append(-1)
                else:
                    input_feature.append(POS_dict[doc[i + j + next].pos_])
            
            if plural == 1:
                input_feature.append(1)
            else:
                input_feature.append(0)

            if i - 1 < 0:
                input_feature.append(bigram_count.get(('<start>', doc[i]), 0))
            else:
                input_feature.append(bigram_count.get((doc[i-1], doc[i]), 0))
            
            if i + 1 >= len(doc):
                input_feature.append(bigram_count.get((doc[i], '<end>'), 0))
            else:
                input_feature.append(bigram_count.get((doc[i], doc[i + 1]), 0))
                
            pps.append((word, (i,pp), input_feature))
    return pps


nlp = spacy.load('en_core_web_sm')
ex = "I recommend visiting the artificial lake in the certer of the city which is surrounded by a park ."
doc = nlp(ex)

print(get_pps(doc, type_list[1], {"a": 0, "the": 1, "null": 2, "an":0}))

[('null', (5, 'the artificial lake'), [6, 0, 1, 6, 0, 0, 0]), ('null', (11, 'the city which is surrounded by a park'), [1, 6, -1, -1, 0, 0, 0])]


In [210]:
import numpy as np
def input_feature_transform(input_feature):
    a = np.array(input_feature)
    return a.T

In [211]:
article = ["a", "the", "null"]
prep = ["about", "along", "among", "around", "as", "at", "beside", "besides", "between", "by", "down", "during",
"except", "for", "from", "in", "inside", "into", "of", "off", "on", "onto", "outside", "over", 
"through", "to", "toward", "towards", "under", "underneath", "until", "up", "upon", "with", "within", "without"]

article_dict = {"a": 0, "the": 1, "null": 2}
prep_dict = {}
for i in range(len(prep)):
    prep_dict[prep[i]] = i

In [266]:
def generate_input(dict_input, size):
    X_article = []
    Y_article = []
    for i in range(size):
        Y_article.append([])
    for i in correct:
        extract = get_pps(nlp(i), type_list[1], dict_input)
        for j in extract:
            X_article.append(j[2])
            word = j[0]
            index = dict_input.get(word, -1)
            for k in Y_article:
                if k == index:
                    k.append(1)
                else:
                    k.append(-1)
    X_input = input_feature_transform(X_article)
    Y_input = input_feature_transform(Y_article)
    return X_input, Y_input

X_input, Y_input = generate_input(prep_dict, len(prep_dict))

print(X_input.shape)
print(Y_input.shape)

(7, 3101)
(3101, 36)


In [247]:
from scipy.optimize import minimize
class linear_classifier:
    def __init__(self, X, Y, name) -> None:
        self.u = np.random.rand(7,1)
        self.lamb = 1e-4
        self.X = X
        self.Y = Y
        self.name = name
    def predict(self, X):
        # self.X = input[0:]
        # self.Y = input[1:]
        compute = self.u.T @ X
        return compute
        
    def update(self):
        def loss_function(x, *args) -> float:
            compute = x.T @ self.X
            compute[compute > 0] = 1
            compute[compute <= 0] = -1
            condition = compute*self.Y

            for i in range(condition.shape[0]):
                if condition[i] < -1:
                    compute[i] = -4*condition[i]
                elif condition[i] > 1:
                    compute[i] = 0
                else:
                    compute[i] = (1 - condition[i])**2
            L = np.sum(compute)/condition.shape[0] + self.lamb*(x.T @ x)  
            return L
        self.u = minimize(loss_function, self.u, method="BFGS", tol=1e-6).x.T

a = np.random.rand(1,4)
b = np.random.rand(1,4)
a = linear_classifier(X_input, Y_input[:,5], "of")
print(a.u.T)
a.update()
print(a.u.T)

[[0.99358788 0.83471854 0.21820746 0.74632216 0.14668985 0.44692038
  0.70193668]]
[6.67331466e-05 5.34698884e-05 4.11029078e-05 2.64650811e-05
 5.34067438e-05 7.22289334e-06 4.90728553e-05]


In [271]:
classifier = []
for i in range(len(prep_dict)):
    curr = linear_classifier(X_input, Y_input[:, i], prep[i])
    curr.update()
    classifier.append(curr)

In [259]:
def test(sentence):
    _input = nlp(sentence) 
    fix = {}
    chunk = get_pps(_input, type_list[1], prep_dict)
    for i in chunk:
        result = []
        location = i[1][0]
        X_input = input_feature_transform(i[2])
        for j in classifier:
            predict_val = j.predict(X_input)
            result.append((predict_val, j.name))
        sorted_result = sorted(result, key=lambda x: x[0],reverse=True)  
        if sorted_result[0][0] - sorted_result[1][0] > 0.7:
            if sorted_result[0][0] < 0:
                fix[location] = "~"
            else:
                fix[location] = sorted_result[0][1]
    correct = []         
    for i in range(len(_input)):
        if i not in fix:
            correct.append(_input[i].text)
        else:
            correct.append(fix[i])
    return ' '.join(correct)
        
test("They usually lack language and can not open a debate")

'They usually lack language and can not open a debate'

In [272]:
import csv
# Function to read the first column (incorrect sentences) from a CSV
def read_incorrect_sentences_from_csv(file_path):
    sentences = []
    with open(file_path, mode='r', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header row if there is one
        for row in reader:
            sentences.append((row[1], row[0]))  # Assuming the first column has the correct sentences and the second has the incorrect sentences
    return sentences

# Function to write corrected sentences to a CSV
def write_corrected_sentences_to_csv(file_path, sentences):
    with open(file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Target Sentence', 'Corrected Sentence'])  # Header row
        for sentence in sentences:
            writer.writerow(sentence)

# Main process
input_csv_path = 'ABC_train_gold_bea19_300samples_copy.csv'  # Path to your input CSV file
output_csv_path = 'target.csv'  # Path to your output CSV file

sentences = read_incorrect_sentences_from_csv(input_csv_path)
print("csv read")
corrected_sentences = []
for incorrect_sentence, target_sentence in sentences:
    corrected = test(incorrect_sentence)
    corrected_sentences.append((target_sentence, corrected))

write_corrected_sentences_to_csv(output_csv_path, corrected_sentences)
print("csv write")


csv read
csv write


In [253]:
%pip install python-Levenshtein
%pip install jiwer
%pip install rouge

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Dell\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


  Attempting uninstall: click
    Found existing installation: click 8.0.3
    Uninstalling click-8.0.3:
      Successfully uninstalled click-8.0.3
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Dell\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Dell\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [273]:
import csv
import numpy as np
from Levenshtein import distance as levenshtein_distance
from jiwer import wer
from rouge import Rouge

# Function to read sentences from a CSV file
def read_sentences_from_csv(file_path):
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        target_sentences = []
        correct_sentences = []
        for row in reader:
            target_sentences.append(row['Target Sentence'])
            correct_sentences.append(row['Corrected Sentence'])
    return target_sentences, correct_sentences

# Compute Exact Match Accuracy
def exact_match_accuracy(targets, corrects):
    exact_matches = [1 for target, correct in zip(targets, corrects) if target == correct]
    return sum(exact_matches) / len(targets)

# Calculate Levenshtein Distance
def average_levenshtein_distance(targets, corrects):
    distances = [levenshtein_distance(target, correct) for target, correct in zip(targets, corrects)]
    return np.mean(distances)

# Determine Word Error Rate
def average_wer(targets, corrects):
    error_rates = [wer(correct, target) for target, correct in zip(targets, corrects)]
    return np.mean(error_rates)

# Compute ROUGE Score
def compute_rouge(targets, corrects):
    rouge = Rouge()
    scores = rouge.get_scores(targets, corrects, avg=True)
    return scores

# Main function to compute metrics
def main(csv_file_path):
    target_sentences, correct_sentences = read_sentences_from_csv(csv_file_path)
    
    exact_match = exact_match_accuracy(target_sentences, correct_sentences)
    levenshtein_dist = average_levenshtein_distance(target_sentences, correct_sentences)
    word_error_rate = average_wer(target_sentences, correct_sentences)
    rouge_scores = compute_rouge(target_sentences, correct_sentences)
    
    print(f"Exact Match Accuracy: {exact_match}")
    print(f"Average Levenshtein Distance: {levenshtein_dist}")
    print(f"Average Word Error Rate: {word_error_rate}")
    print("ROUGE Scores:", rouge_scores)

# Replace 'sentences.csv' with the path to your actual CSV file
main('target.csv')

Exact Match Accuracy: 0.37606837606837606
Average Levenshtein Distance: 5.726495726495727
Average Word Error Rate: 0.0924992397749662
ROUGE Scores: {'rouge-1': {'r': 0.9406156471258732, 'p': 0.9335302402287936, 'f': 0.9361704547597999}, 'rouge-2': {'r': 0.8532181727904609, 'p': 0.843944912502821, 'f': 0.8479967324143409}, 'rouge-l': {'r': 0.9384486079823948, 'p': 0.9313521698220254, 'f': 0.9339985608652032}}
